In [ ]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
from scipy.interpolate import NearestNDInterpolator
from math import * 
plt.rcParams['figure.figsize'] = [12, 9]



### #1

In [ ]:
plt.rcParams['figure.figsize'] = [18,9]

img_o = cv2.imread("./inputs/P1/the_magnificent_eleven_1.png",0)
img = img_o.copy()
dft = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)
magnitude, angle = cv2.cartToPolar(dft_shift[:, :, 0],dft_shift[:, :, 1])
magnitude = np.log(magnitude)
magnitude_log = magnitude.copy()

Skip = 15
magnitude[magnitude.shape[0]//2,:magnitude.shape[1]//2-Skip] = 0
magnitude[magnitude.shape[0]//2,magnitude.shape[1]//2+Skip:] = 0
magnitude[:magnitude.shape[0]//2-Skip,magnitude.shape[1]//2] = 0
magnitude[magnitude.shape[0]//2+Skip:,magnitude.shape[1]//2] = 0

i_magnitude = np.exp(magnitude)
i_dft_shift = cv2.polarToCart(i_magnitude,angle)
i_dft_shift = np.stack([i_dft_shift[0],i_dft_shift[1]],axis=2)
i_dft_shift = np.fft.ifftshift(i_dft_shift)
i_dft = cv2.idft(i_dft_shift, flags=cv2.DFT_SCALE)
out4 = cv2.magnitude(i_dft[:,:,0],i_dft[:,:,1])

plt.subplot(2,2,1)
plt.imshow(img_o,cmap="gray")
plt.axis("off")
plt.title("Input image")

plt.subplot(2,2,3)
plt.imshow(magnitude_log,cmap="gray")
plt.axis("off")

plt.subplot(2,2,2)
plt.imshow(out4.astype(np.uint8),cmap="gray")
plt.axis("off")
plt.title("Remove periodic noise")

plt.subplot(2,2,4)
plt.imshow(magnitude,cmap="gray")
plt.axis("off")

plt.tight_layout()
plt.show()

plt.rcParams['figure.figsize'] = [12,6]
plt.subplot(1,2,1)
result = cv2.medianBlur(out4.astype(np.uint8),3)
plt.imshow(result,cmap="gray")
plt.title("Median bluring")
plt.axis("off")

plt.subplot(1,2,2)
xp = [0, 16, 128, 255]
fp = [0, 32, 128, 255]
x = np.arange(256)
table = np.interp(x, xp, fp).astype('uint8')
img = cv2.LUT(result.astype('uint8'), table)

plt.imshow(img,cmap="gray")
plt.axis("off")
plt.title("Contrast Stretching(Piecewise Linear)")

plt.tight_layout()
plt.show()



plt.subplot(1,2,1)

plt.imshow(img_o,cmap="gray")
plt.title("Input image")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(img,cmap="gray")
plt.axis("off")
plt.title("Image enhancement")

plt.tight_layout()
plt.show()

#plt.imsave("./outputs/P01/1.png",cv2.hconcat([img_o,img]),cmap="gray")


### #2

In [ ]:
plt.rcParams['figure.figsize'] = [18,9]

img_o = cv2.imread("./inputs/P1/the_magnificent_eleven_2.png",0)
img = img_o.copy()


sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
sharpen = cv2.filter2D(img, -1, sharpen_kernel)

plt.subplot(1,3,1)
plt.imshow(img_o,cmap="gray")
plt.axis("off")
plt.title("Input image")

plt.subplot(1,3,2)
plt.imshow(sharpen,cmap="gray")
plt.axis("off")
plt.title("Deblured(Using Laplacian)")


plt.subplot(1,3,3)
xp = [0, 16, 128, 255]
fp = [0, 48, 128, 255]
x = np.arange(256)
table = np.interp(x, xp, fp).astype('uint8')
img = cv2.LUT(sharpen.astype('uint8'), table)

kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
img = cv2.filter2D(src=img, ddepth=-1, kernel=kernel)

plt.imshow(img,cmap="gray")
plt.axis("off")
plt.title("Contrast Stretching(Piecewise Linear) & Sharp")
plt.tight_layout()
plt.show()

#plt.imsave("./outputs/P01/2.png",cv2.hconcat([img_o,img]),cmap="gray")

### #3

In [ ]:
plt.rcParams['figure.figsize'] = [18,9]

img_o = cv2.imread("./inputs/P1/the_magnificent_eleven_3.png",0)
img = img_o.copy()



median = cv2.fastNlMeansDenoising(img,h=21)
plt.subplot(1,3,1)
plt.imshow(img_o,cmap="gray")
plt.axis("off")
plt.title("Input image")

plt.subplot(1,3,2)
plt.imshow(median,cmap="gray")
plt.axis("off")
plt.title("Non-local means denoised")


plt.subplot(1,3,3)


adjusted = median.copy()
VALUE = adjusted[50,50]
for _j in range(adjusted.shape[1]):
    for _i in range(adjusted.shape[0]):
                
        if adjusted[_i,_j] < 135 :
            if _j > 600 :
                if adjusted[_i,_j] < 120 :
                    break
            else:
                break 
        adjusted[_i,_j] = VALUE

img = adjusted
plt.imshow(img,cmap="gray")
plt.axis("off")
plt.title("Local manipulation on upper half")
plt.tight_layout()
plt.show()

#plt.imsave("./outputs/P01/3.png",cv2.hconcat([img_o,img]),cmap="gray")

### #4

In [ ]:
def gammaCorrection(src, gamma):
    invGamma = 1 / gamma

    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)

    return cv2.LUT(src, table)

plt.rcParams['figure.figsize'] = [18,9]
h,w = 2,2
i = 1
img_o = cv2.imread("./inputs/P1/the_magnificent_eleven_4.png",0)
img = img_o.copy()
plt.subplot(h,w,i)
plt.imshow(img_o,cmap="gray")
plt.axis("off")
plt.title("Input image")
i+=1



plt.subplot(h,w,i)
res = cv2.fastNlMeansDenoising(img,h=11)
plt.imshow(res,cmap="gray")
plt.axis("off")
plt.title("Non-local means denoised")
i+=1


plt.subplot(h,w,i)
gamma = 1.5
adjusted = gammaCorrection(res, gamma=gamma)
plt.imshow(adjusted,cmap="gray")
plt.axis("off")
plt.title("Gamma Correction")
i+=1

plt.subplot(h,w,i)
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
adjusted = cv2.filter2D(src=adjusted, ddepth=-1, kernel=kernel)
plt.imshow(adjusted,cmap="gray")
plt.axis("off")
plt.title("Deblur with laplacian")
i+=1


plt.tight_layout()
plt.show()

#plt.imsave("./outputs/P01/4.png",cv2.hconcat([img_o,adjusted]),cmap="gray")


### #5

In [ ]:

plt.rcParams['figure.figsize'] = [18,9]
h,w = 2,2
i = 1
img_o = cv2.imread("./inputs/P1/the_magnificent_eleven_5.png",0)
img = img_o.copy()
plt.subplot(h,w,i)
plt.imshow(img_o,cmap="gray")
plt.axis("off")
plt.title("Input image")
i+=1


plt.subplot(h,w,i)
res = cv2.fastNlMeansDenoising(img,h=17)
plt.imshow(res,cmap="gray")
plt.axis("off")
plt.title("Non-local means denoised")
i+=1


plt.subplot(h,w,i)
gamma = 1.5
adjusted = gammaCorrection(res, gamma=gamma)

plt.imshow(adjusted,cmap="gray")
plt.axis("off")
plt.title("Gamma Correction")
i+=1

plt.subplot(h,w,i)
sep = 187
adjusted_1 = adjusted.copy()
adjusted_1[:sep,:] = adjusted_1[sep+2,:]
tmp = cv2.blur(adjusted_1[:sep,:],(30,sep))
adjusted_1[:sep,:] = tmp
plt.imshow(adjusted_1,cmap="gray")


plt.axis("off")
plt.title("Local manipulation with averaging on upper half")
i+=1

plt.tight_layout()
plt.show()

#plt.imsave("./outputs/P01/5.png",cv2.hconcat([img_o,adjusted_1]),cmap="gray")

### #6

In [ ]:

plt.rcParams['figure.figsize'] = [18,9]
h,w = 2,2
i = 1
img_o = cv2.imread("./inputs/P1/the_magnificent_eleven_6.png",0)
img = img_o.copy()
plt.subplot(h,w,i)
plt.imshow(img_o,cmap="gray")
plt.axis("off")
plt.title("Input image")
i+=1


plt.subplot(h,w,i)
res = cv2.fastNlMeansDenoising(img,h=17)
plt.imshow(res,cmap="gray")
plt.axis("off")
plt.title("Non-local means denoised")
i+=1


plt.subplot(h,w,i)
gamma = 1.5
adjusted = gammaCorrection(res, gamma=gamma)
plt.imshow(adjusted,cmap="gray")
plt.axis("off")
plt.title("Gamma Correction")
i+=1


plt.subplot(h,w,i)
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
adjusted_1 = cv2.filter2D(src=adjusted, ddepth=-1, kernel=kernel)

plt.imshow(adjusted_1,cmap="gray")
plt.axis("off")
plt.title("Deblur using laplacian")
i+=1

plt.tight_layout()
plt.show()

#plt.imsave("./outputs/P01/6.png",cv2.hconcat([img_o,adjusted_1]),cmap="gray")